In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from action import Action
# from PDDL import PDDL_Parser
from PDDLz3 import PDDL_Parser_z3
import sys, pprint
from collections import OrderedDict
from typing import List, Tuple, Dict, Iterable
import re, copy, json
import itertools
import z3
from skill_classes import EffectTypePDDL, SkillPDDL
from utils import product_dict, nested_list_replace, get_atoms, get_all_objects, condition_str2objects
from scoping import scope
import time

In [3]:
def pvars2objects(pvars):
    objs = condition_str2objects(map(str,pvars))
    objs = [s.strip() for s in objs]
    objs = sorted(list(set(objs)))
    return objs

In [4]:
def scope_file(domain, problem):
    parser = PDDL_Parser_z3()
    parser.parse_domain(domain)
    parser.parse_problem(problem)
    with open(problem, "r") as f:
        if ":metric" in f.read():
            raise ValueError(":metric makes the parser or scoper freeze")

    skill_list = parser.get_skills()

    # This below block converts all the domain's goals to z3
    goal_cond = parser.get_goal_cond()

    # This below block converts all the domain's initial conditions to z3
    init_cond_list = parser.get_init_cond_list()

    # Run the scoper on the constructed goal, skills and initial condition
    rel_pvars, rel_skills = scope(goals=goal_cond, skills=skill_list, start_condition=init_cond_list)

    all_pvars = []
    for s in skill_list:
        all_pvars.extend(get_atoms(s.precondition))
        all_pvars.extend(s.params)
    all_objects = pvars2objects(all_pvars)
    rel_objects = pvars2objects(rel_pvars)
    irrel_objects = [x for x in all_objects if x not in rel_objects]
    irrel_pvars = [str(x) for x in all_pvars if str(x) not in map(str, all_pvars)]

    # return rel_objects, irrel_objects
    return rel_pvars, irrel_pvars


In [5]:
results = OrderedDict()

In [5]:
domprobs = [
    ("examples/existential-taxi/taxi-domain.pddl", "./examples/existential-taxi/prob-02.pddl"),
    ("examples/settlers/Settlers.pddl", "examples/settlers/pfile1"),
    ("examples/settlers/Settlers.pddl","examples/settlers/pfile16"),
    ("examples/Satellite/Numeric/adlmetricSat.pddl","examples/Satellite/Numeric/pfile9"),
    ("examples/Satellite/Numeric/metricSat.pddl","examples/Satellite/Numeric/pfile9"), #Bad freeze
    ("examples/Satellite/Strips/adlSat.pddl", "examples/Satellite/Strips/pfile11"),
    ("examples/Rovers/Numeric/NumRover.pddl", "examples/Rovers/Numeric/pfile13"),
    ("examples/freecell/free.pddl", "examples/freecell/pfile13"),
    ("examples/dinner/dinner.pddl", "examples/dinner/pb1.pddl"),
    ("examples/dwr/dwr.pddl", "examples/dwr/pb2.pddl"),
    ("examples/blocksworld/blocksworld.pddl", "examples/blocksworld/pb4.pddl")
]
for dom, prob in domprobs:
    print(f"({dom}, {prob})")
    start_time = time.time()
    try:
        rel, irrel = scope_file(dom, prob)
        r = irrel
    except Exception as e:
        r = e
    duration = time.time() - start_time
    r = (duration, r)
    print(r)
    results[(dom, prob)] = r
    

(examples/existential-taxi/taxi-domain.pddl, ./examples/existential-taxi/prob-02.pddl)
taxi time
(1.0511178970336914, [])


NameError: name 'results' is not defined

In [6]:
with open("./scope_results.json", "r") as f:
    json.dump(results, f, indent=4)

FileNotFoundError: [Errno 2] No such file or directory: './scope_results.json'

In [7]:
5

5